In [ ]:
!pip install wandb==0.13.4

Found existing installation: wandb 0.13.4
Uninstalling wandb-0.13.4:
  Successfully uninstalled wandb-0.13.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached wandb-0.13.4-py2.py3-none-any.whl (1.9 MB)


In [ ]:
import wandb
import torch
import os
import math
import time

In [ ]:
!rm -rf /content/wandb

In [ ]:
wandb.login(host='https://api.wandb.ai' , relogin=True)
run = wandb.init(entity='demonstrations', project='system_metrics', tags =['13.4'])

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: frds (demonstrations). Use `wandb login --relogin` to force relogin


In [ ]:

# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
xx = xx.to(device)

In [ ]:

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
    
)


In [ ]:
model = model.to(device)

In [ ]:
run.watch(model)
for i in range(10000):
    out = model(xx)
    out = torch.dot(out,out.T)
    #time.sleep(1)
    run.log({'loss':x})
run.finish()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)
  after removing the cwd from sys.path.


In [ ]:
with wandb.init(entity='demonstrations', project='system_metrics', tags =['13.4']) as run:
  for i in range(10000):
    out = model(xx)
    out = torch.dot(out,out.T)
    #time.sleep(1)
    run.log({'loss':x})
run.finish()

In [ ]:

def save_model(model, model_dir):
    #logger.info("Saving the model")
    path = os.path.join(model_dir, "model.pth")
    os.makedirs(model_dir,exist_ok=True)
    torch.save(model.cpu(), path)
    model_artifact = wandb.Artifact(name='model.pth', type='model')
    model_artifact.add_file(path)
    run.log_artifact(model_artifact)
    run.finish()
    return

save_model(model, 'model_dir')

In [ ]:
run